# **Preprocessing**

Import and Load Libraries

In [ ]:
import pandas as pd
from tabulate import tabulate
import numpy as np
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import TruncatedSVD
import os
import joblib
import json

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


from google.colab import drive
drive.mount('/content/drive')

Import Files

In [ ]:
# Define the correct folder path
base_path = "/content/drive/MyDrive/Capstone Project/Data/"

# Load feature datasets
df = pd.read_csv(base_path + "Hospital_Inpatient_Discharges.csv", dtype={"Birth Weight":"str"})

Basic Preprocessing

In [ ]:
# Remove unnecessary features
df =  df.drop(df.columns[[0, 2, 13, 14, 15, 16, 17, 18, 19, 22, 28, 29, 30, 33]], axis=1)

In [ ]:
# Remove rows that do not contain hospital information
df = df.dropna(subset=["Hospital County"])

In [ ]:
# Fixing issue of one facility with two IDs
facility_counts = df.groupby(["Facility Name", "Permanent Facility Id"]).size().reset_index(name="Count") #Count ["Facility Name", "Permanent Facility Id"] pairs
multi_id_facilities = facility_counts.groupby("Facility Name")["Permanent Facility Id"].nunique() #Find facilities with multiple IDs
multi_id_facilities = multi_id_facilities[multi_id_facilities > 1].index
for facility in multi_id_facilities: #Replace less frequent with more frequent IDs
    sub_df = facility_counts[facility_counts["Facility Name"] == facility]
    most_frequent_id = sub_df.loc[sub_df["Count"].idxmax(), "Permanent Facility Id"]
    least_frequent_id = sub_df.loc[sub_df["Count"].idxmin(), "Permanent Facility Id"]
    df.loc[(df["Facility Name"] == facility) & (df["Permanent Facility Id"] == least_frequent_id), "Permanent Facility Id"] = most_frequent_id

In [ ]:
## Zip Code - 3 digits
# Removing facilities without a zip code
hospitals_with_zip = df.groupby('Permanent Facility Id')['Zip Code - 3 digits'].transform(lambda x: x.notna().any())
df = df[hospitals_with_zip]

# Replacing missing values
df["Zip Code - 3 digits"] = df.groupby("Permanent Facility Id")["Zip Code - 3 digits"].transform(lambda x: x.ffill().bfill())

# Fixing issue of one facility with multiple Zip Codes
zip_counts = df.groupby(["Permanent Facility Id", "Zip Code - 3 digits"]).size().reset_index(name="Count") #Count ["Permanent Facility Id", "Zip Code - 3 digits"] pairs
multi_zip_facilities = zip_counts.groupby("Permanent Facility Id")["Zip Code - 3 digits"].nunique()
multi_zip_facilities = multi_zip_facilities[multi_zip_facilities > 1].index
for facility in multi_zip_facilities:
    sub_df = zip_counts[zip_counts["Permanent Facility Id"] == facility]
    most_frequent_zip = sub_df.loc[sub_df["Count"].idxmax(), "Zip Code - 3 digits"]
    df.loc[df["Permanent Facility Id"] == facility, "Zip Code - 3 digits"] = most_frequent_zip

In [ ]:
# Handle "120 +" in Length of Stay
df["Length of Stay"] = df["Length of Stay"].replace("120 +", "121")
df["Length of Stay"] = pd.to_numeric(df["Length of Stay"])

In [ ]:
# Handling missing values
df = df.dropna(subset=["APR Risk of Mortality"])
df["Payment Typology 2"] = df["Payment Typology 2"].fillna("Covered by 1st Payment")

Handle Outliers

In [ ]:
Q1 = df["Total Charges"].quantile(0.25)
Q3 = df["Total Charges"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5* IQR
outliers = df[(df["Total Charges"] < lower_bound) | (df["Total Charges"] > upper_bound)]
df = df[(df["Total Charges"] >= lower_bound) & (df["Total Charges"] <= upper_bound)]

outliers_count = [["", "Value"], ["Number of outliers found:", len(outliers)]]
print(tabulate(outliers_count, headers="firstrow", tablefmt="rounded_outline"))

╭───────────────────────────┬─────────╮
│                           │   Value │
├───────────────────────────┼─────────┤
│ Number of outliers found: │  205352 │
╰───────────────────────────┴─────────╯


Mappings for Shiny App

In [ ]:
# Facility mapping
facility_mapping = df[["Permanent Facility Id", "Facility Name"]].drop_duplicates()
facility_mapping_dict = facility_mapping.set_index("Permanent Facility Id")["Facility Name"].to_dict()

In [ ]:
# Illness mapping
illness_mapping = df[["APR MDC Code", "APR MDC Description"]].drop_duplicates()
illness_mapping_dict = illness_mapping.set_index("APR MDC Code")["APR MDC Description"].to_dict()

In [ ]:
# Drop mapped data
df = df.drop(["Facility Name", "APR MDC Description"], axis=1)

In [ ]:
# Age mapping
age_mapping = {"0 to 17": 1, "18 to 29": 2, "30 to 49": 3, "50 to 69": 4, "70 or Older": 5}

In [ ]:
# Risk of mortality and severity of illness mapping
risk_severity_mapping = {"Minor": 1, "Moderate": 2, "Major": 3, "Extreme": 4}

Encode Ordinal Features

In [ ]:
# Create ordinal/int features dataframe
ordinal_df = pd.DataFrame()
ordinal_df['Age Group'] = df['Age Group'].map(age_mapping)
ordinal_df["Length of Stay"] = df["Length of Stay"]
ordinal_df['Severity of Illness'] = df['APR Severity of Illness Description'].map(risk_severity_mapping)
ordinal_df['Risk of Mortality'] = df['APR Risk of Mortality'].map(risk_severity_mapping)
ordinal_df["Total Charges"] = df["Total Charges"]

In [ ]:
# Scale features
scaler = StandardScaler()
scaled_ordinal = scaler.fit_transform(ordinal_df)
scaled_ordinal_df = pd.DataFrame(scaled_ordinal, columns=ordinal_df.columns)

One-Hot-Encoding (Sparse Output)

In [ ]:
print("Starting One-Hot-Encoding...")
df["Emergency Department Indicator"] = df["Emergency Department Indicator"].map({"Y": 1, "N": 0})
sparse_emergency_indicator = csr_matrix(df["Emergency Department Indicator"].values).T

Starting One-Hot-Encoding...


In [ ]:
cols_to_encode = ["Hospital County", "Permanent Facility Id", "Zip Code - 3 digits", "Gender",
                  "Race", "Ethnicity", "Type of Admission", "Patient Disposition", "APR MDC Code",
                  "APR Medical Surgical Description", "Payment Typology 1", "Payment Typology 2"]

for col in cols_to_encode:
    df[col] = df[col].astype(str)

In [ ]:
# Initialize and fit the encoding
encoder = OneHotEncoder(sparse_output=True)
sparse_encoded_features = encoder.fit_transform(df[cols_to_encode])

feature_names = encoder.get_feature_names_out(cols_to_encode)

one_hot_sparse_df = hstack([sparse_emergency_indicator, sparse_encoded_features])
one_hot_sparse_df = one_hot_sparse_df.tocsr()

print("One-Hot-Encoding completed!")

One-Hot-Encoding completed!


In [ ]:
# Shape of the dataset after encoding
print("Encoding Summary")
print(tabulate([
    ["Ordinal Categorical Features", f"{scaled_ordinal_df.shape[0]:,}", f"{scaled_ordinal_df.shape[1]:,}"],
    ["One-Hot-Encoded Features", f"{one_hot_sparse_df.shape[0]:,}", f"{one_hot_sparse_df.shape[1]:,}"]
], headers=["Feature Type", "Rows", "Features"], tablefmt="rounded_outline"))

Encoding Summary
╭──────────────────────────────┬───────────┬────────────╮
│ Feature Type                 │ Rows      │   Features │
├──────────────────────────────┼───────────┼────────────┤
│ Ordinal Categorical Features │ 2,132,821 │          5 │
│ One-Hot-Encoded Features     │ 2,132,821 │        397 │
╰──────────────────────────────┴───────────┴────────────╯


Dimensionality Reduction

In [ ]:
# Gather a small sample of data to fit the SVD and get the optimal number of components
sample_size = int(one_hot_sparse_df.shape[0] * 0.05) #Roughly 5% of the data

In [ ]:
# Calculate proportions and sample count for each age group
age_proportions = ordinal_df['Age Group'].value_counts(normalize=True)
age_sample_sizes = (age_proportions * sample_size).astype(int)

In [ ]:
# Take stratified sample
random_indices = []
for age_group in age_proportions.index:
    group_indices = np.where(ordinal_df['Age Group'] == age_group)[0]
    group_sample_size = age_sample_sizes[age_group]

    # Make sure we don't sample more than available
    group_sample_size = min(group_sample_size, len(group_indices))

    # Sample indices
    if len(group_indices) > 0:
        group_random_indices = np.random.choice(group_indices,
                                                size=group_sample_size,
                                                replace=False)
        random_indices.extend(group_random_indices)

In [ ]:
# Shuffle the combined indices
random_indices = np.array(random_indices)
np.random.shuffle(random_indices)
print(f"Using sample size of {len(random_indices):,} rows.")
data_sample = one_hot_sparse_df[random_indices]

Using sample size of 106,638 rows.


In [ ]:
# Estimate components using the sample
data_sample = one_hot_sparse_df[random_indices]
print("Starting SVD with a smaller sample...")
n_components_initial = min(data_sample.shape[1] - 1, 200)
svd_sample = TruncatedSVD(n_components=n_components_initial, random_state=42)
svd_sample.fit(data_sample)

Starting SVD with a smaller sample...


TruncatedSVD(n_components=200, random_state=42)

In [ ]:
# Calculate number of components needed for 95% variance
cumulative_variance_ratio = np.cumsum(svd_sample.explained_variance_ratio_)
n_components_95 = np.argmax(cumulative_variance_ratio >= 0.95) + 1

In [ ]:
# Add smaller buffer since we're using a larger sample
n_components_95 = int(n_components_95 * 1.05)  # 5% buffer
print(f"Estimated components needed: {n_components_95}")
print("SVD on a smaller sample completed!")

Estimated components needed: 134
SVD on a smaller sample completed!


In [ ]:
# Fit final SVD with the determined number of components
print("Starting SVD on the full dataset...")
svd = TruncatedSVD(n_components=n_components_95, random_state=42)
transformed_one_hot = svd.fit_transform(one_hot_sparse_df)

print(f"{n_components_95} components explained {svd.explained_variance_ratio_.sum():.4f} of the variance.")

print("SVD on full dataset completed!")

Starting SVD on the full dataset...
134 components explained 0.9546 of the variance.
SVD on full dataset completed!


Combine Datasets

In [ ]:
# Concat datasets
transformed_one_hot_df = pd.DataFrame(
    transformed_one_hot,
    columns=[f'SVD_Component_{i+1}' for i in range(transformed_one_hot.shape[1])]
)

final_df = pd.concat([scaled_ordinal_df, transformed_one_hot_df], axis=1)

In [ ]:
# Shape of the dataset after dimensionality deduction
print("Cleaned Data Summary")
transformed_data_size = final_df.shape
print(tabulate([["Rows", transformed_data_size[0]], ["Features", transformed_data_size[1]]], tablefmt="rounded_outline"))

Cleaned Data Summary
╭──────────┬─────────╮
│ Rows     │ 2132821 │
│ Features │     139 │
╰──────────┴─────────╯


Save Dataset

In [ ]:
# Output paths
drive_path = "/content/drive/My Drive/Capstone Project/"

In [ ]:
# Mappings path
mappings_path = os.path.join(drive_path, "Code/ShinyApp/Mappings/")

# Save mappings to JSON files
unique_df = ordinal_df.drop(["Total Charges"], axis=1)
unique_df = ordinal_df.copy()
unique_ordinal_mapping = {col: unique_df[col].unique().tolist() for col in unique_df.columns}
unique_encoded_mapping = {col: df[col].unique().tolist() for col in cols_to_encode}
unique_values = {**unique_ordinal_mapping, **unique_encoded_mapping}
for col in unique_values:
    if isinstance(unique_values[col], list):
        unique_values[col] = [v for v in unique_values[col] if v is not None and str(v).strip() != ""]

mappings = {"facility_mapping.json": facility_mapping_dict,
            "illness_mapping.json": illness_mapping_dict,
            "age_mapping.json": age_mapping,
            "risk_severity_mapping.json": risk_severity_mapping,
            "unique_values.json": unique_values}

for file_name, mapping in mappings.items():
    full_path = os.path.join(mappings_path, file_name)
    with open(full_path, "w", encoding="utf-8") as file:
        json.dump(mapping, file, indent=4)
    print(f"File saved to {full_path}")

File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Mappings/facility_mapping.json
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Mappings/illness_mapping.json
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Mappings/age_mapping.json
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Mappings/risk_severity_mapping.json
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Mappings/unique_values.json


In [ ]:
# Preprocessing components path
preprocessing_components_path = os.path.join(drive_path, "Code/ShinyApp/Preprocessing Components/")

# Save preprocessing components
preprocessing_components = {"scaler.joblib": scaler,
                            "encoder.joblib": encoder,
                            "svd.joblib": svd}

for file_name, component in preprocessing_components.items():
    full_path = os.path.join(preprocessing_components_path, file_name)
    joblib.dump(component, full_path)
    print(f"File saved to {full_path}")

File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Preprocessing Components/scaler.joblib
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Preprocessing Components/encoder.joblib
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Preprocessing Components/svd.joblib


In [ ]:
# Feature names path
features_path = os.path.join(drive_path, "Code/ShinyApp/Features/")

features = {"categorical_columns.json": cols_to_encode,
            "ordinal_columns.json": ordinal_df.columns.tolist()}

for file_name, feature in features.items():
    full_path = os.path.join(features_path, file_name)
    with open(full_path, "w", encoding="utf-8") as file:
        json.dump(feature, file, indent=4)
    print(f"File saved to {full_path}")

File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Features/categorical_columns.json
File saved to /content/drive/My Drive/Capstone Project/Code/ShinyApp/Features/ordinal_columns.json


In [ ]:
# Save cleaned data
full_path = os.path.join(drive_path, "Data/cleaned_data.csv")
final_df.to_csv(full_path, index=False)
print(f"File saved to {full_path}")

File saved to /content/drive/My Drive/Capstone Project/Data/cleaned_data_witout_raceetnicity.csv
